In [ ]:
import pandas as pd
df_test=pd.read_csv("/content/cccc_evaluation_dataset.csv")
del df_test["Unnamed: 0"]
df_test = df_test.rename(columns={"Label": "label"})
df_test

,Complaint,label
0,Someone used my card to buy $500 worth of elec...,fraud
1,I received a text asking to verify my card det...,fraud
2,"A $1,200 flight booking was made with my card ...",fraud
3,My card was charged $300 at a gas station in a...,fraud
4,"I got a call from ""card security"" asking for m...",fraud
...,...,...
245,A hotel resort fee of $30/day was not mentione...,billing_issue
246,"My payment of 180wasprocessedincorrectly,leadi...",billing_issue
247,"I was billed $10 for a replacement PIN, which ...",billing_issue
248,"A $7 ""transaction fee"" was applied to a paymen...",billing_issue


In [ ]:
df_test.label.value_counts()

,count
label,
trouble_using_card,51
fraud,50
customer_service,50
billing_issue,50
features,49


In [ ]:
label_map_test = {
    "billing_issue": 0,
    "customer_service": 1,
    "fraud": 3,
    "trouble_using_card": 4,
    "features":2
}
df_test["label"] = df_test["label"].map(label_map_test)

In [ ]:
df_test

,Complaint,label
0,Someone used my card to buy $500 worth of elec...,3
1,I received a text asking to verify my card det...,3
2,"A $1,200 flight booking was made with my card ...",3
3,My card was charged $300 at a gas station in a...,3
4,"I got a call from ""card security"" asking for m...",3
...,...,...
245,A hotel resort fee of $30/day was not mentione...,0
246,"My payment of 180wasprocessedincorrectly,leadi...",0
247,"I was billed $10 for a replacement PIN, which ...",0
248,"A $7 ""transaction fee"" was applied to a paymen...",0


In [ ]:
df_test.label.value_counts()

,count
label,
4,51
3,50
1,50
0,50
2,49


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_name = "kkkkkjjjjjj/credit-card-complaint-classifier"  # Replace with your actual model name

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model.eval()  # Set to evaluation mode


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/917 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from torch.utils.data import DataLoader, Dataset

class ComplaintDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label)
        }

test_dataset = ComplaintDataset(df_test['Complaint'].tolist(), df_test['label'].tolist(), tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16)


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Evaluation
print(classification_report(all_labels, all_preds, target_names=["billing", "customer_service", "features", "fraud", "trouble_using_card"]))


                    precision    recall  f1-score   support

           billing       0.88      0.92      0.90        50
  customer_service       0.92      0.92      0.92        50
          features       0.80      0.67      0.73        49
             fraud       0.88      1.00      0.93        50
trouble_using_card       0.90      0.88      0.89        51

          accuracy                           0.88       250
         macro avg       0.88      0.88      0.88       250
      weighted avg       0.88      0.88      0.88       250



In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(all_labels, all_preds)
print("Overall Accuracy: {:.2f}%".format(accuracy * 100))


Overall Accuracy: 88.00%
